In [ ]:
import datetime
import os
import pandas as pd

from google.cloud import bigquery
from google.oauth2 import service_account
from dotenv import load_dotenv

In [ ]:
def set_credential(project_id, gcp_credentials):
	credentials = service_account.Credentials.from_service_account_file(gcp_credentials)
	client = bigquery.Client(project = project_id, credentials = credentials)

	return client

In [ ]:
def get_dates(client, project_id, table_id):
	sql = f"SELECT MIN(`date`), MAX(`date`)FROM `{project_id}.base.{table_id}`;"
	query = client.query(sql)
	result = query.result()
	row = next(result)

	return row[0], row[1]

In [ ]:
def get_query_dates(min_date, max_date):
	dates = []
	month_first_day = min_date.replace(day = 1)

	while month_first_day <= max_date:
		if month_first_day.month == 12:
			next_month_first_day = datetime.date(month_first_day.year + 1, 1, 1)
		else:
			next_month_first_day = datetime.date(month_first_day.year, month_first_day.month + 1, 1)

		day_query = next_month_first_day - datetime.timedelta(days = 1)

		if day_query >= min_date and day_query <= max_date:
			dates.append(day_query)

		month_first_day = next_month_first_day

	return dates

In [ ]:
def get_data(client, project_id, table_id, date):
	date = date.strftime('%Y-%m-%d')
	
	sql = f"""
		SELECT
			SAFE_CAST(LEFT(phone, 2) AS INT64) AS ddd,
			COUNT(phone) as qty
		FROM `{project_id}.base.{table_id}`
		WHERE
			`date` = '{date}'
			AND fn_is_active IS TRUE
			AND plan_provider_id IS NOT NULL
			AND REGEXP_CONTAINS(phone, r'^[0-9]{{11}}$')
		GROUP BY 1
		ORDER BY 1;
	"""
	
	query = client.query(sql)
	result = query.result()

	return result

In [ ]:
load_dotenv()

In [ ]:
project_id = os.getenv('PROJECT_ID')
table_id = os.getenv('TABLE_ID')
gcp_credentials = os.getenv('GCP_CREDENTIALS')

In [ ]:
client = set_credential(project_id, gcp_credentials)

In [ ]:
min_date, max_date = get_dates(client, project_id, table_id)

In [ ]:
query_dates = get_query_dates(min_date, max_date)

In [ ]:
for date in query_dates:
	data = []
	result = get_data(client, project_id, table_id, date)
	
	for row in result:
		data.append(dict(row))
	
	month = date.strftime('%Y%m')
	df = pd.DataFrame(data)
	df['month'] = month
	df = df.iloc[:, [2, 0, 1]]

	df.to_csv(f"sources/customer-data/{month}.csv", index = False)